# Smart-Turn Dictionary Fine-Tuning Pipeline

In [ ]:
# !pip install requests pandas tqdm


First convert senteneces to csv

In [ ]:
import pandas as pd

sentences = [
    "मैंने वो loop condition fix कर दी है।",
    "सागर सर ने नयी policy announce की है।",
    "आज API response time बहुत fast है।",
    "मैंने वो Docker container restart कर दिया है।",
    "अगर हम logic change करेंगे तो result सही आएगा।",
    "मैंने वो null pointer exception handle कर लिया है।",
    "वो front-end design approve हो गया है।",
    "मुझे लगता है कि हमें React version upgrade करना चाहिए।",
    "मैंने वो JSON file parse कर ली है।",
    "वो integration test pass हो गया है।",
    "मैंने अपनी shift timing change करवा ली है।",
    "वो conference room book हो गया है।",
    "आज office में attendance biometric से हो रही है।",
    "मैंने वो expense report submit कर दी है।",
    "वो ID card print होके आ गया है।",
    "मुझे लगता है कि हमें team meeting करनी चाहिए।",
    "मैंने अपना laptop bag pack कर लिया है।",
    "वो holiday list notice board पर लगा दी गयी है।",
    "सूरत में आज मौसम बहुत cloudy है।",
    "मैंने mouse ki battery change कर दी है।",
    "वो monitor screen साफ़ कर ली है।",
    "मैंने keyboard shortcut set कर लिया है।",
    "वो HDMI port अब काम कर रहा है।",
    "मैंने system virus scan कर लिया है।",
    "वो external hard drive connect हो गयी है।",
    "मुझे लगता है कि webcam driver update करना पड़ेगा।",
    "मैंने अपना headset mute कर दिया है।",
    "वो LAN cable replace कर दी गयी है।",
    "मैंने printer paper tray भर दिया है।",
    "आज auto मिलने में बहुत दिक्कत हुई थी।",
    "मैंने वो parcel courier से भेज दिया है।",
    "वो street light अब जल रही है।",
    "मुझे लगता है कि आज बारिश होगी।",
    "मैंने अपना lunch box धो लिया है।",
    "वो रास्ता construction की वजह से बंद है।",
    "मैंने Adajan से नयी bike खरीदी है।",
    "वो database connection string update हो गयी है।",
    "मैंने sham को Vesu जाने का plan बनाया है।",
    "आज office का coffee machine खराब है।",
    "मैंने वो notification disable कर दिया है।"
]
df = pd.DataFrame({"id": list(range(1, len(sentences)+1)), "text": sentences})
path = "/content/mix_csv/true.csv"
df.to_csv(path, index=False, encoding="utf-8-sig")
print(path)


/content/mix_csv/true.csv


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Step 2: converts to tts using Cartesia

In [ ]:
import os
import requests
import pandas as pd
from tqdm import tqdm

# =============================
# CONFIG
# =============================
CARTESIA_API_KEY = "sk_car_dYNwXUgcK7eFqQGG9DgK6d"  # <-- put your key here
CSV_FOLDER = "/content/mix_csv"
OUTPUT_DIR = "mix_audio"
MODEL_ID = "sonic-3-preview"

# 🔊 VOICE POOL (auto-rotated)
VOICE_POOL = [
    "791d5162-d5eb-40f0-8189-f19db44611d8",
    "6303e5fb-a0a7-48f9-bb1a-dd42c216dc5d",
    "bec003e2-3cb3-429c-8468-206a393c67ad",
    "95d51f79-c397-46f9-b49a-23763d3eaa2d",
    "faf0731e-dfb9-4cfc-8119-259a79b27e12",
    "c4494312-1667-43f0-b09e-2100fd1c0585",
    "4fbe1f86-a023-4d44-a7e7-8708f62f35cf",
    "700d1ee3-a641-4018-ba6e-899dcadc9e2b",
    "28ca2041-5dda-42df-8123-f58ea9c3da00",
    "56e35e2d-6eb6-4226-ab8b-9776515a7094",
    "209d9a43-03eb-40d8-a7b7-51a6d54c052f",
    "47f3bbb1-e98f-4e0c-92c5-5f0325e1e206",
    "be79f378-47fe-4f9c-b92b-f02cefa62ccf",
    "9b953e7b-86a8-42f0-b625-1434fb15392b",
    "55e2a153-c61e-4784-85c8-e954cb22fe29"
]

os.makedirs(OUTPUT_DIR, exist_ok=True)

# =============================
# CARTESIA TTS FUNCTION
# =============================
def generate_tts(text, voice_id, output_path):
    url = "https://api.cartesia.ai/tts/bytes"

    headers = {
        "X-API-Key": CARTESIA_API_KEY,
        "Cartesia-Version": "2025-04-16",
        "Content-Type": "application/json"
    }

    def request(voice):
        return requests.post(
            url,
            headers=headers,
            json={
                "model_id": MODEL_ID,
                "voice": {"mode": "id", "id": voice},
                "transcript": text,
                "output_format": {
                    "container": "wav",
                    "encoding": "pcm_f32le",
                    "sample_rate": 44100
                }
            }
        )

    response = request(voice_id)

    if response.status_code == 200:
        with open(output_path, "wb") as f:
            f.write(response.content)
        return

    print(f"⚠️ Voice failed ({voice_id}), retrying fallback")

    fallback_voice = "bec003e2-3cb3-429c-8468-206a393c67ad"
    retry = request(fallback_voice)

    if retry.status_code == 200:
        with open(output_path, "wb") as f:
            f.write(retry.content)
    else:
        print("❌ Failed:", retry.text)

# =============================
# PROCESS ALL CSV FILES
# =============================
audio_counter = 1

csv_files = sorted(
    f for f in os.listdir(CSV_FOLDER)
    if f.lower().endswith(".csv")
)

assert csv_files, "❌ No CSV files found"

for csv_name in csv_files:
    csv_path = os.path.join(CSV_FOLDER, csv_name)
    print(f"\n📄 Processing {csv_name}")

    df = pd.read_csv(csv_path)
    assert "text" in df.columns, f"'text' column missing in {csv_name}"

    csv_base = os.path.splitext(csv_name)[0]

    for _, row in tqdm(df.iterrows(), total=len(df)):
        text = str(row["text"]).strip()
        if not text:
            continue

        voice_id = VOICE_POOL[audio_counter % len(VOICE_POOL)]

        output_file = os.path.join(
            OUTPUT_DIR,
            f"{csv_base}_audio_{audio_counter}.wav"
        )

        generate_tts(text, voice_id, output_file)
        audio_counter += 1

print(f"\n✅ DONE! All WAV files saved in '{OUTPUT_DIR}'")


📄 Processing true.csv


100%|██████████| 40/40 [00:40<00:00,  1.02s/it]


✅ DONE! All WAV files saved in 'mix_audio'


In [ ]:
!zip -r tts_audio_n.zip tts_audio_n
from google.colab import files
files.download("tts_audio_n.zip")


step 3: convert to parquet

In [ ]:
!ls /content/mix_audios//true_audio | head
!ls /content/mix_audios//false_audio | head


ls: cannot access '/content/mix_audios//true_audio': No such file or directory
ls: cannot access '/content/mix_audios//false_audio': No such file or directory


In [ ]:
!rm -rf audio train.csv
!mkdir audio


In [ ]:
!cp /content/smart_turn_dataset/true_audio/*.wav audio/
!cp /content/smart_turn_dataset/false_audio/*.wav audio/


In [ ]:
!ls audio | wc -l


10000


In [ ]:
import os, pandas as pd, random

random.seed(42)

rows = []

for folder, label in [("true_audio", True), ("false_audio", False)]:
    files = [f for f in os.listdir(f"/content/smart_turn_dataset/{folder}") if f.endswith(".wav")]
    for f in files:
        rows.append({
            "id": os.path.splitext(f)[0],
            "audio": f"audio/{f}",
            "endpoint_bool": label,
            "language": "hin",
            "midfiller": False,
            "endfiller": False,
            "synthetic": False,
            "spoken_text": None,
            "dataset": "custom_v1"
        })

# 🔀 shuffle rows
random.shuffle(rows)

df = pd.DataFrame(rows)
df.to_csv("train.csv", index=False)

print("CSV rows:", len(df))
print(df["endpoint_bool"].head(20).tolist())



CSV rows: 4000
[True, True, False, False, False, False, False, False, False, False, False, False, False, False, True, True, True, False, False, False]


In [ ]:
import pandas as pd, os

df = pd.read_csv("train.csv")
missing = [f for f in df["audio"] if not os.path.exists(f)]
print("Missing files:", len(missing))


Missing files: 0


In [ ]:
from huggingface_hub import whoami
whoami()


{'type': 'user',
 'id': '691aaace252905727f1e377b',
 'name': 'vrushil',
 'fullname': 'Davra',
 'email': 'vrushildavra1234@gmail.com',
 'emailVerified': True,
 'canPay': False,
 'billingMode': 'prepaid',
 'periodEnd': 1769904000,
 'isPro': False,
 'avatarUrl': 'https://cdn-avatars.huggingface.co/v1/production/uploads/no-auth/n1kTg5CgBW38ws2_Dq3cs.png',
 'orgs': [],
 'auth': {'type': 'access_token',
  'accessToken': {'displayName': 'API key 1',
   'role': 'write',
   'createdAt': '2026-01-02T09:48:19.396Z'}}}

In [ ]:
# from datasets import load_dataset, Audio

# ds = load_dataset("csv", data_files="train.csv")
# ds = ds.cast_column("audio", Audio(sampling_rate=16000))

# ds.push_to_hub("vrushil/smart-turn-data-10000", private=False)


Uploading the dataset shards:   0%|          | 0/4 [00:00<?, ? shards/s]

Map:   0%|          | 0/2500 [00:00<?, ? examples/s]

Creating parquet from Arrow format:   0%|          | 0/25 [00:00<?, ?ba/s]

Processing Files (0 / 0)      : |          |  0.00B /  0.00B            

New Data Upload               : |          |  0.00B /  0.00B            

...

Creating parquet from Arrow format:   0%|          | 0/25 [00:00<?, ?ba/s]

Processing Files (0 / 0)      : |          |  0.00B /  0.00B            

New Data Upload               : |          |  0.00B /  0.00B            

                              :   1%|          | 4.07MB /  472MB            

CommitInfo(commit_url='https://huggingface.co/datasets/vrushil/smart-turn-data-10000/commit/6d220c145cd9a13ebbd81a088b856d60e215dd0a', commit_message='Upload dataset', commit_description='', oid='6d220c145cd9a13ebbd81a088b856d60e215dd0a', pr_url=None, repo_url=RepoUrl('https://huggingface.co/datasets/vrushil/smart-turn-data-10000', endpoint='https://huggingface.co', repo_type='dataset', repo_id='vrushil/smart-turn-data-10000'), pr_revision=None, pr_num=None)

In [ ]:
!pip install -U torchcodec


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.1/2.1 MB 25.7 MB/s eta 0:00:00


In [ ]:
labels = ds["train"]["endpoint_bool"][:200]
print(labels)


[True, True, False, False, False, False, False, False, False, False, False, False, False, False, True, True, True, False, False, False, True, True, True, False, False, True, False, True, False, True, False, True, True, False, True, False, False, False, False, False, True, True, False, True, True, False, False, False, True, False, False, False, True, False, True, False, False, True, True, True, False, True, True, False, False, False, True, False, True, False, False, False, False, False, True, False, False, True, False, False, False, False, False, True, False, False, True, False, False, True, False, True, True, True, False, False, False, True, False, True, False, False, True, True, True, False, True, False, True, True, True, False, True, True, False, True, False, True, False, True, False, True, True, False, False, True, True, False, False, False, False, False, False, False, True, True, False, True, True, False, False, True, False, False, False, True, True, False, True, True, False, False

if already have dataset in hugging face so do this step(fatch dataset)

In [ ]:
from huggingface_hub import login
login()


In [ ]:
from datasets import load_dataset

dataset = load_dataset(
    "miraiminds/smart-turn-hindi-dataset-14k",
    token=True   # 🔥 this replaces use_auth_token=True
)



README.md:   0%|          | 0.00/617 [00:00<?, ?B/s]

data/train-00000-of-00010.parquet:   0%|          | 0.00/445M [00:00<?, ?B/s]

data/train-00001-of-00010.parquet:   0%|          | 0.00/438M [00:00<?, ?B/s]

data/train-00002-of-00010.parquet:   0%|          | 0.00/443M [00:00<?, ?B/s]

data/train-00003-of-00010.parquet:   0%|          | 0.00/440M [00:00<?, ?B/s]

...

data/train-00006-of-00010.parquet:   0%|          | 0.00/443M [00:00<?, ?B/s]

data/train-00007-of-00010.parquet:   0%|          | 0.00/436M [00:00<?, ?B/s]

data/train-00008-of-00010.parquet:   0%|          | 0.00/430M [00:00<?, ?B/s]

data/train-00009-of-00010.parquet:   0%|          | 0.00/436M [00:00<?, ?B/s]

Generating train split:   0%|          | 0/14000 [00:00<?, ? examples/s]

In [ ]:
dataset.save_to_disk("smart_turn_hf_local")


Saving the dataset (0/10 shards):   0%|          | 0/14000 [00:00<?, ? examples/s]

step 4:install Required Libraries for Smart-Turn Training

In [ ]:
!pip uninstall -y torch torchvision torchaudio torchao transformers


Found existing installation: torch 2.9.0+cu126
Uninstalling torch-2.9.0+cu126:
  Successfully uninstalled torch-2.9.0+cu126
Found existing installation: torchvision 0.24.0+cu126
Uninstalling torchvision-0.24.0+cu126:
  Successfully uninstalled torchvision-0.24.0+cu126
Found existing installation: torchaudio 2.9.0+cu126
Uninstalling torchaudio-2.9.0+cu126:
  Successfully uninstalled torchaudio-2.9.0+cu126
Found existing installation: torchao 0.10.0
Uninstalling torchao-0.10.0:
  Successfully uninstalled torchao-0.10.0
Found existing installation: transformers 4.57.3
Uninstalling transformers-4.57.3:
  Successfully uninstalled transformers-4.57.3


In [ ]:
!pip install torch==2.3.1 torchvision==0.18.1 torchaudio==2.3.1 --index-url https://download.pytorch.org/whl/cu121


Looking in indexes: https://download.pytorch.org/whl/cu121
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 780.9/780.9 MB 635.7 kB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.0/7.0 MB 125.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.4/3.4 MB 118.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 23.7/23.7 MB 105.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 823.6/823.6 kB 60.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 14.1/14.1 MB 131.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 731.7/731.7 MB 717.2 kB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 410.6/410.6 MB 3.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 121.6/121.6 MB 8.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.5/56.5 MB 14.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 124.2/124.2 MB 8.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 196.0/1

In [ ]:
pip install torchao==0.7.0 transformers==4.40.2 datasets==2.18.0


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 138.0/138.0 kB 5.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.1/4.1 MB 2.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.0/9.0 MB 95.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 510.5/510.5 kB 17.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 170.9/170.9 kB 6.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.6/3.6 MB 73.3 MB/s eta 0:00:00
  Attempting uninstall: fsspec
    Found existing installation: fsspec 2025.3.0
    Uninstalling fsspec-2025.3.0:
      Successfully uninstalled fsspec-2025.3.0
  Attempting uninstall: tokenizers
    Found existing installation: tokenizers 0.22.1
    Uninstalling tokenizers-0.22.1:
      Successfully uninstalled tokenizers-0.22.1
  Attempting uninstall: datasets
    Found existing installation: datasets 4.0.0
    Uninstalling datasets-4.0.0:
      Successfully uninstalled datasets-4.0.0
ERROR: pip's dependency resolver doe

In [ ]:
import torch, torchvision, torchaudio, torchao, transformers

print("torch:", torch.__version__)
print("torchvision:", torchvision.__version__)
print("torchaudio:", torchaudio.__version__)
print("torchao:", torchao.__version__)
print("transformers:", transformers.__version__)


The cache for model files in Transformers v4.22.0 has been updated. Migrating your old cache. This is a one-time only operation. You can interrupt this and resume the migration later on by calling `transformers.utils.move_cache()`.


0it [00:00, ?it/s]

torch: 2.3.1+cu121
torchvision: 0.18.1+cu121
torchaudio: 2.3.1+cu121
torchao: 0.7.0
transformers: 4.40.2


In [ ]:
!pip install -U onnx onnxruntime onnxruntime-tools


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 18.1/18.1 MB 60.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 17.4/17.4 MB 18.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 212.7/212.7 kB 9.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 46.0/46.0 kB 2.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 55.5/55.5 kB 2.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 86.8/86.8 kB 4.0 MB/s eta 0:00:00


In [ ]:
!pip uninstall -y transformers peft accelerate safetensors
!pip install -U \
  transformers==4.41.2 \
  peft==0.10.0 \
  accelerate==0.29.3 \
  safetensors==0.4.3


Found existing installation: transformers 4.40.2
Uninstalling transformers-4.40.2:
  Successfully uninstalled transformers-4.40.2
Found existing installation: peft 0.18.0
Uninstalling peft-0.18.0:
  Successfully uninstalled peft-0.18.0
Found existing installation: accelerate 1.12.0
Uninstalling accelerate-1.12.0:
  Successfully uninstalled accelerate-1.12.0
Found existing installation: safetensors 0.7.0
Uninstalling safetensors-0.7.0:
  Successfully uninstalled safetensors-0.7.0
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.8/43.8 kB 2.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.1/9.1 MB 38.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 199.1/199.1 kB 9.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 297.6/297.6 kB 14.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 35.9 MB/s eta 0:00:00


In [ ]:
!pip install -q modal


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 744.3/744.3 kB 17.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 40.4/40.4 kB 1.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 285.4/285.4 kB 11.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 456.8/456.8 kB 9.9 MB/s eta 0:00:00


step 5 : upload two file benchmark and logger


step 6 : Train model

In [ ]:
import os
from dataclasses import dataclass
from typing import List, Dict, Union

import numpy as np
import onnx
import torch
import wandb
from onnxruntime.quantization import quantize_static, CalibrationDataReader, QuantType, quant_pre_process, \
    QuantFormat, CalibrationMethod
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, confusion_matrix
from torch import nn
from torch.export import Dim
from torch.nn.functional import softmax
from torch.utils.data import Dataset
from transformers import WhisperFeatureExtractor, WhisperPreTrainedModel, WhisperConfig
# noinspection PyProtectedMember
from transformers.models.whisper.modeling_whisper import WhisperEncoder
# noinspection PyProtectedMember
from transformers.trainer import Trainer
from transformers.trainer_callback import TrainerCallback
from transformers.trainer_utils import IntervalStrategy
from transformers.training_args import TrainingArguments

from benchmark import benchmark
from datasets import load_dataset, concatenate_datasets, load_from_disk
from logger import log, log_model_structure, log_dataset_statistics, log_dependencies, ProgressLoggerCallback

CONFIG = {
    "run_name_prefix": "v3.1",
    "base_model_name": "ARTPARK-IISc/whisper-tiny-vaani-hindi",

    "datasets_training": ["miraiminds/smart-turn-hindi-dataset-14k"],
    "datasets_test": ["miraiminds/smart-turn-hindi-dataset-14k"],

    "learning_rate": 5e-5,
    "num_epochs": 5,
    "train_batch_size": 8,
    "eval_batch_size": 8,
    "warmup_ratio": 0.2,
    "weight_decay": 0.01,

    "eval_steps": 500,
    "save_steps": 500,
    "logging_steps": 100,

    "onnx_opset_version": 18,
    "calibration_dataset_size": 128,
}



class SmartTurnV3Model(WhisperPreTrainedModel):
    def __init__(self, config: WhisperConfig):
        super().__init__(config)

        config.max_source_positions = 400
        self.encoder = WhisperEncoder(config)

        # Use the encoder's hidden size
        hidden_size = config.d_model

        self.pool_attention = nn.Sequential(
            nn.Linear(hidden_size, 256),
            nn.Tanh(),
            nn.Linear(256, 1)
        )

        self.classifier = nn.Sequential(
            nn.Linear(hidden_size, 256),
            nn.LayerNorm(256),
            nn.GELU(),
            nn.Dropout(0.1),
            nn.Linear(256, 64),
            nn.GELU(),
            nn.Linear(64, 1)
        )

        # Initialize classifier weights
        for module in self.classifier:
            if isinstance(module, nn.Linear):
                module.weight.data.normal_(mean=0.0, std=0.1)
                if module.bias is not None:
                    module.bias.data.zero_()

        # Initialize attention pooling weights
        for module in self.pool_attention:
            if isinstance(module, nn.Linear):
                module.weight.data.normal_(mean=0.0, std=0.1)
                if module.bias is not None:
                    module.bias.data.zero_()

    def forward(self, input_features, labels=None):
        """
        Forward pass using Whisper encoder only

        Args:
            input_features: Log-mel spectrogram features [batch_size, n_mels, n_frames] - now (batch_size, 80, 800)
            labels: Binary labels for endpointing (1 = complete, 0 = incomplete)
        """
        # Use only the encoder part of Whisper
        encoder_outputs = self.encoder(input_features=input_features)

        hidden_states = encoder_outputs.last_hidden_state

        attention_weights = self.pool_attention(hidden_states)
        attention_weights = softmax(attention_weights, dim=1)
        pooled = torch.sum(hidden_states * attention_weights, dim=1)

        logits = self.classifier(pooled)

        if labels is not None:
            # Calculate positive sample weight based on batch statistics
            pos_weight = ((labels == 0).sum() / (labels == 1).sum()).clamp(min=0.1, max=10.0)
            loss_fct = nn.BCEWithLogitsLoss(pos_weight=pos_weight)
            labels = labels.float()
            loss = loss_fct(logits.view(-1), labels.view(-1))

            probs = torch.sigmoid(logits.detach())
            return {"loss": loss, "logits": probs}

        probs = torch.sigmoid(logits)
        return {"logits": probs}


class CalibrationDataset:
    """Calibration dataset for ONNX quantization with stratified sampling (early-stop)."""

    def __init__(self, dataset, feature_extractor, max_samples):

        log.info("Building calibration dataset...")

        self.feature_extractor = feature_extractor

        ds = dataset.dataset.shuffle(seed=42)
        n = min(max_samples, len(ds))
        subset = ds.select(range(n))
        self.dataset = subset

        label_view = subset.select_columns(["endpoint_bool"])
        labels = label_view["endpoint_bool"]
        pos = sum(1 for v in labels if v)
        neg = len(labels) - pos

        log.info(f"Calibration dataset: {n} samples (positives={pos}, negatives={neg})")

    def __len__(self):
        return len(self.dataset)

    def __getitem__(self, idx):
        sample = self.dataset[idx]

        audio_array = sample["audio"]["array"]
        audio_array = truncate_audio_to_last_n_seconds(audio_array, n_seconds=8)

        inputs = self.feature_extractor(
            audio_array,
            sampling_rate=16000,
            return_tensors="pt",
            padding="max_length",
            max_length=8 * 16000,
            truncation=True,
            do_normalize=True,
        )
        return inputs.input_features.squeeze(0).numpy()


class ONNXCalibrationDataReader(CalibrationDataReader):
    def __init__(self, calibration_dataset):
        self.calibration_dataset = calibration_dataset
        self.iterator = iter(range(len(calibration_dataset)))

    def get_next(self):
        try:
            idx = next(self.iterator)
            input_data = self.calibration_dataset[idx]  # shape (80, 800)
            input_data = np.expand_dims(input_data, axis=0)  # shape (1, 80, 800)
            input_data = input_data.astype(np.float32, copy=False)
            return {"input_features": input_data}
        except StopIteration:
            return None


def export_to_onnx_fp32(model, output_path, config):
    """Export model to ONNX FP32 format"""
    try:
        log.info("Exporting model to ONNX FP32...")

        class ONNXExportWrapper(torch.nn.Module):
            def __init__(self, inner):
                super().__init__()
                self.inner = inner

            def forward(self, input_features):
                out = self.inner(input_features)
                logits = out["logits"] if isinstance(out, dict) else out
                # Output (batch_size, 1) shape (2D) - standard format for single-output models
                batch_size = logits.shape[0]
                return logits.reshape(batch_size, 1)

        export_model = ONNXExportWrapper(model).eval().cpu()

        example_input_b1 = torch.randn(1, 80, 800)
        example_input_b2 = torch.randn(2, 80, 800)

        # Test with both batch size 1 and 2 to ensure consistent output shapes
        with torch.no_grad():
            test_output_1 = export_model(example_input_b1)
            test_output_2 = export_model(example_input_b2)
            assert test_output_1.shape == (1, 1), f"Expected (1, 1), got {test_output_1.shape}"
            assert test_output_2.shape == (2, 1), f"Expected (2, 1), got {test_output_2.shape}"


        torch.onnx.export(
            model=export_model,
            args=(example_input_b2,),
            f=output_path,
            export_params=True,
            opset_version=config["onnx_opset_version"],
            do_constant_folding=False,
            input_names=['input_features'],
            output_names=['logits'],
            dynamic_axes={
                'input_features': {0: 'batch_size'},
                'logits': {0: 'batch_size'}
            },
            verbose=False,
        )

        onnx_model = onnx.load(output_path)
        onnx.checker.check_model(onnx_model)
        log.info(f"FP32 ONNX model saved to {output_path}")

        # Verify the exported model works with batch sizes 1 and 2 and outputs consistent shapes
        import onnxruntime as ort
        session = ort.InferenceSession(output_path)

        example_input_1_np = example_input_b1.numpy().astype(np.float32)
        outputs_1 = session.run(None, {'input_features': example_input_1_np})
        assert outputs_1[0].shape == (1, 1), f"Expected (1, 1), got {outputs_1[0].shape}"

        example_input_2_np = example_input_b2.numpy().astype(np.float32)
        outputs_2 = session.run(None, {'input_features': example_input_2_np})
        assert outputs_2[0].shape == (2, 1), f"Expected (2, 1), got {outputs_2[0].shape}"

        log.info("ONNX model verification successful - consistent output shapes for both batch sizes")

        return output_path

    except Exception:
        log.exception("Failed to export to ONNX")
        return None


from onnxruntime.quantization import quantize_static, QuantType, QuantFormat, CalibrationMethod

def quantize_onnx_model(
    onnx_fp32_path: str,
    training_dataset,
    feature_extractor,
    exports_path,
    calibration_dataset_size: int
):
    import os

    assert os.path.exists(onnx_fp32_path), f"Model not found: {onnx_fp32_path}"

    output_path = os.path.join(
        exports_path,
        f"model_int8_static_calib{calibration_dataset_size}.onnx"
    )

    log.info("Building calibration dataset...")
    calibration_dataset = CalibrationDataset(
        training_dataset,
        feature_extractor,
        max_samples=calibration_dataset_size,
    )

    log.info("Invoking quantize_static (NO PREPROCESS, SAFE MODE)...")

    quantize_static(
        model_input=onnx_fp32_path,
        model_output=output_path,
        calibration_data_reader=ONNXCalibrationDataReader(calibration_dataset),
        quant_format=QuantFormat.QDQ,
        activation_type=QuantType.QUInt8,
        weight_type=QuantType.QInt8,
        per_channel=True,
        calibrate_method=CalibrationMethod.Entropy,
        op_types_to_quantize=["Conv", "MatMul", "Gemm"],
    )

    log.info(f"INT8 model saved to: {output_path}")
    return output_path



def load_dataset_at(path: str):
    if path.startswith('/'):
        return load_from_disk(path)["train"]
    else:
        return load_dataset(path)["train"]


def truncate_audio_to_last_n_seconds(audio_array, n_seconds=8, sample_rate=16000):
    max_samples = n_seconds * sample_rate
    if len(audio_array) > max_samples:
        return audio_array[-max_samples:]
    return audio_array


class OnDemandSmartTurnDataset(Dataset):
    def __init__(self, hf_dataset, feature_extractor):
        self.dataset = hf_dataset
        self.feature_extractor = feature_extractor

    def __len__(self):
        return len(self.dataset)

    def __getitem__(self, idx):
        sample = self.dataset[idx]

        audio_array = sample["audio"]["array"]

        audio_array = truncate_audio_to_last_n_seconds(audio_array, n_seconds=8)

        label = 1 if sample["endpoint_bool"] else 0

        inputs = self.feature_extractor(
            audio_array,
            sampling_rate=16000,
            return_tensors="pt",
            padding="max_length",
            max_length=8 * 16000,
            truncation=True,
            do_normalize=True,
        )

        return {
            "input_features": inputs.input_features.squeeze(0),
            "labels": torch.tensor(label, dtype=torch.long),
            "language": sample.get("language", "eng"),
            "dataset": sample.get("dataset", "unknown"),
            "midfiller": sample.get("midfiller", None),
            "endfiller": sample.get("endfiller", None),
        }


@dataclass
class SmartTurnDataCollator:
    def __call__(self, features: List[Dict[str, Union[torch.Tensor, str, None]]]) -> Dict[str, torch.Tensor]:
        input_features = torch.stack([f["input_features"] for f in features])
        labels = torch.stack([f["labels"] for f in features])

        batch = {
            "input_features": input_features,
            "labels": labels,
        }

        if "language" in features[0]:
            batch["language"] = [f["language"] for f in features]
        if "midfiller" in features[0]:
            batch["midfiller"] = [f["midfiller"] for f in features]
        if "endfiller" in features[0]:
            batch["endfiller"] = [f["endfiller"] for f in features]

        return batch


def prepare_datasets_ondemand(feature_extractor, config):
    log.info("Preparing datasets...")

    datasets_training = config["datasets_training"]
    datasets_test = config["datasets_test"]

    training_splits = []
    eval_splits = []
    test_splits = {}

    for dataset_path in datasets_training:
        dataset_name = dataset_path.split("/")[-1]
        log.info(f"Loading dataset '{dataset_name}'...")
        full_dataset = load_dataset_at(dataset_path)

        log.info("  |-> Splitting dataset into train/eval splits...")
        dataset_dict = full_dataset.train_test_split(test_size=0.1, seed=42)
        training_splits.append(dataset_dict["train"])
        eval_splits.append(dataset_dict["test"])

    log.info("Merging datasets...")

    merged_training_dataset = concatenate_datasets(training_splits).shuffle(seed=42)
    merged_eval_dataset = concatenate_datasets(eval_splits)

    log.info("Loading test datasets...")

    for dataset_path in datasets_test:
        dataset_name = dataset_path.split("/")[-1]
        test_dataset = load_dataset_at(dataset_path)
        test_splits[dataset_name] = test_dataset

    merged_test_dataset = concatenate_datasets(test_splits.values()).shuffle(seed=42)

    log.info("Wrapping datasets with OnDemandWhisperDataset...")
    wrapped_training = OnDemandSmartTurnDataset(merged_training_dataset, feature_extractor)
    wrapped_eval = OnDemandSmartTurnDataset(merged_eval_dataset, feature_extractor)
    wrapped_test_splits = {
        name: OnDemandSmartTurnDataset(dataset, feature_extractor)
        for name, dataset in test_splits.items()
    }
    wrapped_test_merged = OnDemandSmartTurnDataset(merged_test_dataset, feature_extractor)

    return {
        "training": wrapped_training,
        "eval": wrapped_eval,
        "test": wrapped_test_splits,
        "test_merged": wrapped_test_merged,
        "raw_datasets": {
            "training": merged_training_dataset,
            "eval": merged_eval_dataset,
            "test": test_splits
        }
    }


def process_predictions(logits):
    """
    Converts raw logits into squeezed probability predictions and binary predictions.
    """
    if np.isnan(logits).any() or not np.isfinite(logits).all():
        raise ValueError("Non-finite or NaN values detected in logits during processing")

    probs = logits.squeeze()
    preds = (probs > 0.5).astype(int)

    return probs, preds


def get_predictions_and_labels(trainer, dataset, metric_key_prefix=None):
    """
    Returns tuple:
        - predictions: Raw prediction output from trainer
        - labels: Ground truth labels
        - probs: Squeezed probability predictions
        - preds: Binary predictions (probs > 0.5)
    """
    predictions = trainer.predict(dataset, metric_key_prefix=metric_key_prefix)

    probs, preds = process_predictions(predictions.predictions)
    labels = predictions.label_ids

    return predictions, labels, probs, preds


class ExternalEvaluationCallback(TrainerCallback):

    def __init__(self, test_datasets, trainer):
        super().__init__()
        self.test_datasets = test_datasets
        self.trainer = trainer

    def on_evaluate(self, args, state, control, **kwargs):
        accuracies = {}
        language_metrics = {}
        midfiller_metrics = {}

        for dataset_name, dataset in self.test_datasets.items():
            predictions, labels, probs, preds = get_predictions_and_labels(
                self.trainer, dataset, f"exttest/{dataset_name}"
            )

            metrics = compute_metrics((probs, labels))

            external_metrics = {
                f"exttest/{dataset_name}_{k}": v
                for k, v in metrics.items()
            }

            external_metrics[f"exttest/{dataset_name}_prob_dist"] = wandb.Histogram(probs)

            external_metrics[f"train/global_step"] = state.global_step

            accuracies[dataset_name] = metrics["accuracy"]

            wandb.log(external_metrics)

            self._process_category_metrics(dataset, probs, labels, preds, language_metrics,
                                           column_name='language', default_value='unknown-error')
            self._process_category_metrics(dataset, probs, labels, preds, midfiller_metrics,
                                           column_name='midfiller', default_value='unknown')

        self._log_category_metrics(language_metrics, 'lang', state.global_step)
        self._log_category_metrics(midfiller_metrics, 'midfiller', state.global_step)

        if accuracies:
            lowest_accuracy = min(accuracies.values())
            lowest_accuracy_dataset = min(accuracies.keys(), key=lambda k: accuracies[k])

            accuracy_values = list(accuracies.values())
            mean_accuracy = sum(accuracy_values) / len(accuracy_values)

            wandb.log({
                "exttest/lowest_accuracy": lowest_accuracy,
                "exttest/lowest_accuracy_dataset": lowest_accuracy_dataset,
                "exttest/mean_accuracy": mean_accuracy,
                "exttest/accuracy_variance": np.var(accuracy_values),
                "train/global_step": state.global_step
            })

            log.info(f"Overall accuracy metrics:")
            log.info(f"  Lowest accuracy across all test datasets: {lowest_accuracy:.4f} ({lowest_accuracy_dataset})")
            log.info(f"  Mean accuracy: {mean_accuracy:.4f}")
            log.info(f"  Accuracy variance: {np.var(accuracy_values):.4f}")

    def _process_category_metrics(self, dataset, probs, labels, preds, category_metrics,
                                  column_name, default_value):
        if hasattr(dataset, 'dataset'):
            underlying_dataset = dataset.dataset
        else:
            underlying_dataset = dataset

        if hasattr(underlying_dataset, 'column_names') and column_name in underlying_dataset.column_names:
            categories = underlying_dataset[column_name]
        else:
            categories = [default_value] * len(dataset)

        for i, category in enumerate(categories):
            category_key = str(category).lower() if category is not None else default_value

            if category_key not in category_metrics:
                category_metrics[category_key] = {
                    'probs': [],
                    'labels': [],
                    'preds': []
                }

            category_metrics[category_key]['probs'].append(probs[i])
            category_metrics[category_key]['labels'].append(labels[i])
            category_metrics[category_key]['preds'].append(preds[i])

    def _log_category_metrics(self, category_metrics, metric_prefix, global_step):
        category_accuracies = {}

        for category, data in category_metrics.items():
            if len(data['labels']) == 0:
                continue

            cat_probs = np.array(data['probs'])
            cat_labels = np.array(data['labels'])

            metrics = compute_metrics((cat_probs, cat_labels))

            category_specific_metrics = {
                f"exttest/{metric_prefix}_{category}_{k}": v
                for k, v in metrics.items()
            }

            category_specific_metrics[f"exttest/{metric_prefix}_{category}_prob_dist"] = wandb.Histogram(cat_probs)
            category_specific_metrics[f"exttest/{metric_prefix}_{category}_sample_count"] = len(cat_labels)
            category_specific_metrics["train/global_step"] = global_step

            category_accuracies[category] = metrics["accuracy"]

            wandb.log(category_specific_metrics)

            log.info(f"{metric_prefix.capitalize()} {category} metrics: accuracy={metrics['accuracy']:.4f}, "
                     f"precision={metrics['precision']:.4f}, recall={metrics['recall']:.4f}, "
                     f"f1={metrics['f1']:.4f}, samples={len(cat_labels)}")

        if category_accuracies:
            min_accuracy = min(category_accuracies.values())
            max_accuracy = max(category_accuracies.values())
            mean_accuracy = sum(category_accuracies.values()) / len(category_accuracies)

            best_category = max(category_accuracies.keys(), key=lambda k: category_accuracies[k])
            worst_category = min(category_accuracies.keys(), key=lambda k: category_accuracies[k])

            summary_metrics = {
                f"exttest/{metric_prefix}_min_accuracy": min_accuracy,
                f"exttest/{metric_prefix}_max_accuracy": max_accuracy,
                f"exttest/{metric_prefix}_mean_accuracy": mean_accuracy,
                f"exttest/{metric_prefix}_accuracy_range": max_accuracy - min_accuracy,
                f"exttest/best_performing_{metric_prefix}": best_category,
                f"exttest/worst_performing_{metric_prefix}": worst_category,
                f"exttest/{metric_prefix}_categories_evaluated": len(category_accuracies),
                "train/global_step": global_step
            }

            if len(category_accuracies) > 1:
                summary_metrics[f"exttest/{metric_prefix}_accuracy_std"] = np.std(list(category_accuracies.values()))

            wandb.log(summary_metrics)

            category_type = metric_prefix.replace('_', ' ')
            log.info(f"{category_type.capitalize()} performance summary:")
            log.info(f"  Best performing {category_type}: {best_category} ({category_accuracies[best_category]:.4f})")
            log.info(
                f"  Worst performing {category_type}: {worst_category} ({category_accuracies[worst_category]:.4f})")
            log.info(f"  Mean accuracy across {category_type}s: {mean_accuracy:.4f}")
            log.info(f"  Accuracy range: {max_accuracy - min_accuracy:.4f}")

        if category_metrics:
            total_samples = sum(len(data['labels']) for data in category_metrics.values())
            distribution_metrics = {
                f"exttest/{metric_prefix}_{category}_percentage": (len(
                    category_metrics[category]['labels']) / total_samples) * 100
                for category in category_metrics.keys()
            }
            distribution_metrics["train/global_step"] = global_step
            wandb.log(distribution_metrics)


def compute_metrics(eval_pred):
    logits, labels = eval_pred

    probs, preds = process_predictions(logits)

    tn, fp, fn, tp = confusion_matrix(labels, preds).ravel()

    return {
        "accuracy": accuracy_score(labels, preds),
        "precision": precision_score(labels, preds, zero_division="warn"),
        "recall": recall_score(labels, preds, zero_division="warn"),
        "f1": f1_score(labels, preds, zero_division="warn"),
        "pred_positives": tp + fp,
        "pred_negatives": tn + fn,
        "true_positives": tp,
        "false_positives": fp,
        "true_negatives": tn,
        "false_negatives": fn,
    }


def final_evaluate(trainer, dataset, split_name):
    log.info(f"Evaluating on {split_name} set...")
    metrics = trainer.evaluate(eval_dataset=dataset)

    predictions, labels, probs, preds = get_predictions_and_labels(trainer, dataset)

    wandb_metrics = {
        f"final/{split_name}_accuracy": metrics["eval_accuracy"],
        f"final/{split_name}_precision": metrics["eval_precision"],
        f"final/{split_name}_recall": metrics["eval_recall"],
        f"final/{split_name}_f1": metrics["eval_f1"],
    }

    wandb.log(wandb_metrics)

    return metrics, predictions


def do_training_run(run_name_suffix: str):
    log_dependencies()

    run_name = CONFIG["run_name_prefix"] + "-" + run_name_suffix

    log.info(f"Starting training run: {run_name}")

    wandb_api_key = os.environ.get("WANDB_API_KEY")
    if not wandb_api_key:
        raise ValueError("WANDB_API_KEY environment variable not set")

    wandb_run = wandb.init(
        project="speech-endpointing",
        name=run_name,
        config=CONFIG
    )

    wandb_run.define_metric(name="exttest/*", step_metric="train/global_step")

    model = SmartTurnV3Model.from_pretrained(CONFIG["base_model_name"], num_labels=1, ignore_mismatched_sizes=True)
    feature_extractor = WhisperFeatureExtractor(chunk_length=8) # 8 seconds

    log_model_structure(model, CONFIG)

    datasets = prepare_datasets_ondemand(feature_extractor, CONFIG)

    training_args = TrainingArguments(
        output_dir=f"/data/output/{run_name}",
        per_device_train_batch_size=CONFIG["train_batch_size"],
        per_device_eval_batch_size=CONFIG["eval_batch_size"],
        num_train_epochs=CONFIG["num_epochs"],
        eval_strategy=IntervalStrategy.STEPS,
        gradient_accumulation_steps=1,
        eval_steps=CONFIG["eval_steps"],
        save_steps=CONFIG["save_steps"],
        logging_steps=CONFIG["logging_steps"],
        load_best_model_at_end=False,
        metric_for_best_model="f1",
        greater_is_better=True,
        learning_rate=CONFIG["learning_rate"],
        warmup_ratio=CONFIG["warmup_ratio"],
        weight_decay=CONFIG["weight_decay"],
        lr_scheduler_type="cosine",
        report_to=["wandb"],
        dataloader_num_workers=6,
        dataloader_prefetch_factor=4,
        dataloader_pin_memory=True,
        tf32=False,
        disable_tqdm=True,
    )

    os.makedirs(training_args.output_dir, exist_ok=True)

    log_dataset_statistics("training", datasets["training"])
    log_dataset_statistics("eval", datasets["eval"])

    for dataset_name, dataset in datasets["test"].items():
        log_dataset_statistics("test_" + dataset_name, dataset)

    trainer = Trainer(
        model=model,
        args=training_args,
        train_dataset=datasets["training"],
        eval_dataset=datasets["eval"],
        compute_metrics=compute_metrics,
        data_collator=SmartTurnDataCollator(),
        callbacks=[
            ProgressLoggerCallback(log_interval=CONFIG["logging_steps"])
        ]
    )

    trainer.add_callback(ExternalEvaluationCallback(
        test_datasets=datasets["test"],
        trainer=trainer
    ))

    log.info("Starting training...")
    trainer.train()

    final_save_path = f"{trainer.args.output_dir}/final_model"
    os.makedirs(final_save_path, exist_ok=True)

    feature_extractor.save_pretrained(final_save_path)
    trainer.save_model(final_save_path)

    export_path = os.path.join(final_save_path, "exports")
    os.makedirs(export_path, exist_ok=True)

    onnx_fp32_path = os.path.join(export_path, "model_fp32.onnx")

    trainer.model.eval().cpu()

    onnx_fp32_model_path = export_to_onnx_fp32(trainer.model, onnx_fp32_path, CONFIG)

    log.info(f"Training and export completed. Models saved to: {final_save_path}")

    wandb.finish()

    return onnx_fp32_model_path


def do_quantization_run(fp32_model_path: str):
    calibration_dataset_size = CONFIG["calibration_dataset_size"]

    log.info(f"Starting quantization run on {fp32_model_path} (calib dataset size {calibration_dataset_size})")

    feature_extractor = WhisperFeatureExtractor(chunk_length=8)  # 8 seconds

    datasets = prepare_datasets_ondemand(feature_extractor, CONFIG)

    parent_dir = os.path.dirname(fp32_model_path)

    quantized_onnx_path = quantize_onnx_model(
        onnx_fp32_path=fp32_model_path,
        training_dataset=datasets["training"],
        feature_extractor=feature_extractor,
        exports_path=parent_dir,
        calibration_dataset_size=calibration_dataset_size
    )

    return quantized_onnx_path


def do_benchmark_run(model_paths: List[str]):
    log.info(f"Benchmarking models: {model_paths}")

    feature_extractor = WhisperFeatureExtractor(chunk_length=8)  # 8 seconds

    dataset = prepare_datasets_ondemand(feature_extractor, CONFIG)["test_merged"]

    for model_path in model_paths:
        model_name = os.path.basename(model_path).replace(".onnx", "")
        benchmark_path = os.path.join(os.path.dirname(model_path), "benchmarks")
        os.makedirs(benchmark_path, exist_ok=True)

        benchmark(
            onnx_path=model_path,
            run_description=model_name,
            dataset=dataset,
            limit=None,
            markdown_output=f"{benchmark_path}/{model_name}.md",
            batch_size=256
        )

In [ ]:
import os
os.environ["WANDB_API_KEY"] = "02210449feefa3757d8f30b62329cc7fb8e4a1f7"


In [ ]:
# from datasets import load_dataset

# ds = load_dataset("vrushil/smart-turn-hindi-dataset")["train"]

# print(ds[0])


In [ ]:
onnx_fp32_path = do_training_run("hf-custom-hindi")
print("FP32 model:", onnx_fp32_path)

2026-01-04 12:48:23 - endpointing_training 	| --- INSTALLED PYTHON PACKAGES ---
INFO:endpointing_training:--- INSTALLED PYTHON PACKAGES ---
2026-01-04 12:48:28 - endpointing_training 	| Package                                  Version
---------------------------------------- --------------------
absl-py                                  1.4.0
accelerate                               0.29.3
access                                   1.1.10.post3
affine                                   2.4.0
aiofiles                                 24.1.0
aiohappyeyeballs                         2.6.1
aiohttp                                  3.13.2
aiosignal                                1.4.0
aiosqlite                                0.22.0
alabaster                                1.0.0
albucore                                 0.0.24
albumentations                           2.0.8
ale-py                                   0.11.2
alembic                                  1.17.2
altair                         

eval/accuracy,█▇▁▆
eval/f1,█▇▁▆
eval/false_negatives,▂▁█▂
eval/false_positives,▅█▁█
eval/loss,█▁▄▂
eval/precision,▅▁█▁
eval/pred_negatives,▂▁█▂
eval/pred_positives,▇█▁▇
eval/recall,▇█▁▇
eval/runtime,█▃▅▁
+56,...


config.json: 0.00B [00:00, ?B/s]

model.safetensors:   0%|          | 0.00/151M [00:00<?, ?B/s]

Some weights of SmartTurnV3Model were not initialized from the model checkpoint at ARTPARK-IISc/whisper-tiny-vaani-hindi and are newly initialized: ['model.classifier.0.bias', 'model.classifier.0.weight', 'model.classifier.1.bias', 'model.classifier.1.weight', 'model.classifier.4.bias', 'model.classifier.4.weight', 'model.classifier.6.bias', 'model.classifier.6.weight', 'model.pool_attention.0.bias', 'model.pool_attention.0.weight', 'model.pool_attention.2.bias', 'model.pool_attention.2.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
Some weights of SmartTurnV3Model were not initialized from the model checkpoint at ARTPARK-IISc/whisper-tiny-vaani-hindi and are newly initialized because the shapes did not match:
- model.encoder.embed_positions.weight: found shape torch.Size([1500, 384]) in the checkpoint and torch.Size([400, 384]) in the model instantiated
You should probably TRAIN this model on a down-stream task t

...

2026-01-04 13:30:25 - endpointing_training 	| Completed epoch 5/5
INFO:endpointing_training:Completed epoch 5/5
2026-01-04 13:30:25 - endpointing_training 	| Training completed successfully in 41.6 minutes (2494 seconds)
INFO:endpointing_training:Training completed successfully in 41.6 minutes (2494 seconds)
Some non-default generation parameters are set in the model config. These should go into a GenerationConfig file (https://huggingface.co/docs/transformers/generation_strategies#save-a-custom-decoding-strategy-with-your-model) instead. This warning will be raised to an exception in v4.41.
Non-default generation parameters: {'max_length': None}
2026-01-04 13:30:25 - endpointing_training 	| Exporting model to ONNX FP32...
INFO:endpointing_training:Exporting model to ONNX FP32...


{'train_runtime': 2493.8987, 'train_samples_per_second': 25.262, 'train_steps_per_second': 3.158, 'train_loss': 0.23680680720011393, 'epoch': 5.0}


/usr/local/lib/python3.12/dist-packages/torch/onnx/utils.py:1547: OnnxExporterWarning: Exporting to ONNX opset version 18 is not supported. by 'torch.onnx.export()'. The highest opset version supported is 17. To use a newer opset version, consider 'torch.onnx.dynamo_export()'. Note that dynamo_export() is in preview. Please report errors with dynamo_export() as Github issues to https://github.com/pytorch/pytorch/issues.
  warnings.warn(
/usr/local/lib/python3.12/dist-packages/transformers/models/whisper/modeling_whisper.py:1159: TracerWarning: Converting a tensor to a Python boolean might cause the trace to be incorrect. We can't record the data flow of Python values, so this value will be treated as a constant in the future. This means that the trace might not generalize to other inputs!
  if input_features.shape[-1] != expected_seq_length:
/usr/local/lib/python3.12/dist-packages/transformers/models/whisper/modeling_whisper.py:701: TracerWarning: Converting a tensor to a Python boolea

eval/accuracy,▁▂▂▃▆▅▆▇██████▇
eval/f1,▁▄▂▃▇▆▇▇███████
eval/false_negatives,█▁█▆▃▃▂▃▃▂▂▂▁▂▁
eval/false_positives,▃█▁▂▃▃▄▂▂▃▃▂▃▃▃
eval/loss,█▃▄▄▂▁▂▁▁▁▂▂▃▃▄
eval/precision,▆▁█▇▆▆▅▇█▆▆█▆▇▆
eval/pred_negatives,▇▁█▇▄▅▃▅▅▄▄▅▃▄▃
eval/pred_positives,▂█▁▂▅▄▆▄▄▅▅▄▆▅▆
eval/recall,▁█▁▃▆▆▇▆▆▇▇▇█▇█
eval/runtime,▄▄▃▅▄▅▄▂▂▁▃▄█▄▇
+56,...


FP32 model: /data/output/v3.1-hf-custom-hindi/final_model/exports/model_fp32.onnx


step:7 Evaluation & Testing

In [ ]:
%%writefile inference.py
import numpy as np
import onnxruntime as ort
from transformers import WhisperFeatureExtractor

ONNX_MODEL_PATH = "/data/output/v3.1-hf-custom-hindi/final_model/exports/model_fp32.onnx"


def build_session(onnx_path):
    so = ort.SessionOptions()
    so.execution_mode = ort.ExecutionMode.ORT_SEQUENTIAL
    so.inter_op_num_threads = 1
    so.graph_optimization_level = ort.GraphOptimizationLevel.ORT_ENABLE_ALL
    return ort.InferenceSession(onnx_path, sess_options=so)

feature_extractor = WhisperFeatureExtractor(chunk_length=8)
session = build_session(ONNX_MODEL_PATH)

def truncate_audio_to_last_n_seconds(audio_array, n_seconds=8, sample_rate=16000):
    """Truncate audio to last n seconds or pad with zeros to meet n seconds."""
    max_samples = n_seconds * sample_rate
    if len(audio_array) > max_samples:
        return audio_array[-max_samples:]
    elif len(audio_array) < max_samples:
        # Pad with zeros at the beginning
        padding = max_samples - len(audio_array)
        return np.pad(audio_array, (padding, 0), mode='constant', constant_values=0)
    return audio_array


def predict_endpoint(audio_array):
    """
    Predict whether an audio segment is complete (turn ended) or incomplete.

    Args:
        audio_array: Numpy array containing audio samples at 16kHz

    Returns:
        Dictionary containing prediction results:
        - prediction: 1 for complete, 0 for incomplete
        - probability: Probability of completion (sigmoid output)
    """

    # Truncate to 8 seconds (keeping the end) or pad to 8 seconds
    audio_array = truncate_audio_to_last_n_seconds(audio_array, n_seconds=8)

    # Process audio using Whisper's feature extractor
    inputs = feature_extractor(
        audio_array,
        sampling_rate=16000,
        return_tensors="np",
        padding="max_length",
        max_length=8 * 16000,
        truncation=True,
        do_normalize=True,
    )

    # Extract features and ensure correct shape for ONNX
    input_features = inputs.input_features.squeeze(0).astype(np.float32)
    input_features = np.expand_dims(input_features, axis=0)  # Add batch dimension

    # Run ONNX inference
    outputs = session.run(None, {"input_features": input_features})

    # Extract probability (ONNX model returns sigmoid probabilities)
    probability = outputs[0][0].item()

    # Make prediction (1 for Complete, 0 for Incomplete)
    prediction = 1 if probability > 0.5 else 0

    return {
        "prediction": prediction,
        "probability": probability,
    }


# Example usage
if __name__ == "__main__":
    # Create a dummy audio array for testing (1 second of random audio)
    dummy_audio = np.random.randn(16000).astype(np.float32)

    result = predict_endpoint(dummy_audio)
    print(f"Prediction: {result['prediction']}")
    print(f"Probability: {result['probability']:.4f}")

Writing inference.py


In [ ]:
%%writefile predict.py
import os
import librosa
import sys
import numpy as np
from inference import predict_endpoint

def process_file(path):
    audio, sr = librosa.load(path, sr=None, mono=True)
    if sr != 16000:
        audio = librosa.resample(audio, orig_sr=sr, target_sr=16000)
    audio = audio.astype(np.float32)

    result = predict_endpoint(audio)
    return result

if len(sys.argv) < 2:
    print("Usage: python predict.py <audio.wav | folder_path>")
    sys.exit(1)

input_path = sys.argv[1]

if os.path.isdir(input_path):
    files = [f for f in os.listdir(input_path) if f.lower().endswith(".wav")]
    print(f"\nProcessing {len(files)} wav files...\n")

    for f in files:
        path = os.path.join(input_path, f)
        result = process_file(path)
        print(f"{f:30s} -> {'Complete' if result['prediction'] else 'Incomplete'} | {result['probability']:.4f}")

else:
    result = process_file(input_path)
    print("\nResult:")
    print("Prediction:", "Complete" if result["prediction"] else "Incomplete")
    print("Probability:", result["probability"])


Writing predict.py


In [ ]:
!python predict.py /content/mix_audio


Processing 40 wav files...

2026-01-04 13:34:08.944863: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:467] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1767533649.176664   32423 cuda_dnn.cc:8579] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1767533649.249356   32423 cuda_blas.cc:1407] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
W0000 00:00:1767533649.720260   32423 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linking the same target more than once.
W0000 00:00:1767533649.720303   32423 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linking the same target more than once.
W0000 00:00:1767533649.720307   32423 computation_placer.c

Step : 8 for real accuracy

In [ ]:
import re

# Get the output from the predict.py execution
predict_output = """

Processing 100 wav files...

2026-01-02 16:41:58.325853: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:467] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
WARNING: All log messages before absl::InitializeLog() is called are written to STDERR
E0000 00:00:1767372118.366323   24529 cuda_dnn.cc:8579] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1767372118.377028   24529 cuda_blas.cc:1407] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
W0000 00:00:1767372118.403476   24529 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linking the same target more than once.
W0000 00:00:1767372118.403514   24529 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linking the same target more than once.
W0000 00:00:1767372118.403523   24529 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linking the same target more than once.
W0000 00:00:1767372118.403529   24529 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linking the same target more than once.
false_audio_2.wav              -> Incomplete | 0.0043
false_audio_52.wav             -> Complete | 0.9958
true_audio_72.wav              -> Complete | 0.9976
true_audio_99.wav              -> Complete | 0.9979
false_audio_34.wav             -> Incomplete | 0.0019
false_audio_22.wav             -> Incomplete | 0.0020
false_audio_44.wav             -> Incomplete | 0.0031
false_audio_56.wav             -> Complete | 0.9180
false_audio_33.wav             -> Complete | 0.9976
false_audio_58.wav             -> Incomplete | 0.0229
true_audio_62.wav              -> Complete | 0.9848
true_audio_78.wav              -> Complete | 0.9979
true_audio_70.wav              -> Complete | 0.9976
false_audio_20.wav             -> Incomplete | 0.0043
true_audio_89.wav              -> Complete | 0.9978
false_audio_19.wav             -> Incomplete | 0.0019
false_audio_4.wav              -> Incomplete | 0.0936
true_audio_98.wav              -> Complete | 0.9974
true_audio_77.wav              -> Complete | 0.9977
false_audio_36.wav             -> Complete | 0.9961
false_audio_57.wav             -> Incomplete | 0.0023
false_audio_37.wav             -> Complete | 0.9962
true_audio_81.wav              -> Complete | 0.9972
true_audio_87.wav              -> Complete | 0.9977
false_audio_39.wav             -> Complete | 0.9550
false_audio_12.wav             -> Incomplete | 0.0020
false_audio_25.wav             -> Complete | 0.9966
true_audio_90.wav              -> Complete | 0.9971
false_audio_13.wav             -> Incomplete | 0.1063
true_audio_86.wav              -> Complete | 0.9976
false_audio_21.wav             -> Incomplete | 0.0021
false_audio_60.wav             -> Incomplete | 0.0020
false_audio_10.wav             -> Incomplete | 0.0019
false_audio_35.wav             -> Incomplete | 0.1085
false_audio_28.wav             -> Incomplete | 0.0022
true_audio_76.wav              -> Complete | 0.9977
false_audio_8.wav              -> Incomplete | 0.0073
false_audio_40.wav             -> Complete | 0.9915
true_audio_96.wav              -> Complete | 0.9952
false_audio_32.wav             -> Complete | 0.9967
true_audio_84.wav              -> Complete | 0.9971
true_audio_79.wav              -> Complete | 0.9977
true_audio_85.wav              -> Complete | 0.9977
true_audio_82.wav              -> Complete | 0.9978
false_audio_38.wav             -> Complete | 0.9757
true_audio_93.wav              -> Complete | 0.9975
false_audio_43.wav             -> Incomplete | 0.0020
false_audio_29.wav             -> Incomplete | 0.0279
false_audio_24.wav             -> Complete | 0.7763
true_audio_100.wav             -> Complete | 0.9977
true_audio_71.wav              -> Complete | 0.9978
false_audio_3.wav              -> Incomplete | 0.0023
true_audio_88.wav              -> Complete | 0.9979
true_audio_74.wav              -> Complete | 0.9978
true_audio_92.wav              -> Complete | 0.9978
false_audio_16.wav             -> Incomplete | 0.0023
false_audio_9.wav              -> Incomplete | 0.0020
false_audio_11.wav             -> Incomplete | 0.0050
true_audio_94.wav              -> Complete | 0.9978
true_audio_63.wav              -> Complete | 0.9973
true_audio_69.wav              -> Complete | 0.9975
true_audio_75.wav              -> Complete | 0.9978
true_audio_61.wav              -> Complete | 0.9975
false_audio_42.wav             -> Incomplete | 0.0084
true_audio_65.wav              -> Complete | 0.9952
false_audio_6.wav              -> Incomplete | 0.0019
false_audio_54.wav             -> Incomplete | 0.0019
false_audio_51.wav             -> Complete | 0.9493
true_audio_67.wav              -> Complete | 0.9978
false_audio_46.wav             -> Complete | 0.9868
false_audio_47.wav             -> Complete | 0.9960
false_audio_45.wav             -> Complete | 0.9966
false_audio_18.wav             -> Complete | 0.9969
false_audio_15.wav             -> Incomplete | 0.0403
true_audio_64.wav              -> Complete | 0.9977
true_audio_97.wav              -> Complete | 0.9974
false_audio_48.wav             -> Incomplete | 0.0044
false_audio_23.wav             -> Incomplete | 0.0498
true_audio_95.wav              -> Complete | 0.9968
true_audio_83.wav              -> Complete | 0.9976
true_audio_91.wav              -> Complete | 0.9964
true_audio_80.wav              -> Complete | 0.9977
true_audio_66.wav              -> Complete | 0.9968
false_audio_55.wav             -> Incomplete | 0.0051
false_audio_31.wav             -> Incomplete | 0.0034
false_audio_17.wav             -> Complete | 0.9979
false_audio_14.wav             -> Incomplete | 0.0020
false_audio_1.wav              -> Incomplete | 0.0233
false_audio_27.wav             -> Incomplete | 0.0019
false_audio_7.wav              -> Incomplete | 0.0041
false_audio_59.wav             -> Complete | 0.8501
false_audio_50.wav             -> Incomplete | 0.0030
true_audio_68.wav              -> Complete | 0.9976
false_audio_49.wav             -> Incomplete | 0.0030
false_audio_41.wav             -> Incomplete | 0.0022
false_audio_26.wav             -> Incomplete | 0.0019
false_audio_5.wav              -> Incomplete | 0.0025
true_audio_73.wav              -> Complete | 0.9978
false_audio_30.wav             -> Incomplete | 0.0022
false_audio_53.wav             -> Complete | 0.9953
"""

predictions = []
for line in predict_output.splitlines():
    match = re.match(r'(.+\.wav)\s+->\s+(Complete|Incomplete)\s+\|\s+([0-9.]+)', line)
    if match:
        filename = match.group(1).strip()
        predicted_label_str = match.group(2).strip()
        probability = float(match.group(3).strip())

        # Determine true label from filename
        true_label_str = 'Complete' if filename.startswith('true_audio_') else 'Incomplete'

        predictions.append({
            'filename': filename,
            'true_label': true_label_str,
            'predicted_label': predicted_label_str,
            'probability': probability
        })


correct_predictions = 0
misclassified_files = []

for pred in predictions:
    if pred['true_label'] == pred['predicted_label']:
        correct_predictions += 1
    else:
        misclassified_files.append(pred)

total_predictions = len(predictions)
accuracy = (correct_predictions / total_predictions) * 100 if total_predictions > 0 else 0

print(f"Total Predictions: {total_predictions}")
print(f"Correct Predictions: {correct_predictions}")
print(f"Incorrect Predictions: {len(misclassified_files)}")
print(f"Accuracy: {accuracy:.2f}%")

if misclassified_files:
    print("\nMisclassified Files:")
    for item in misclassified_files:
        print(f"  - {item['filename']}: True: {item['true_label']}, Predicted: {item['predicted_label']}, Probability: {item['probability']:.4f}")
else:
    print("\nNo misclassified files found!")


Total Predictions: 100
Correct Predictions: 81
Incorrect Predictions: 19
Accuracy: 81.00%

Misclassified Files:
  - false_audio_52.wav: True: Incomplete, Predicted: Complete, Probability: 0.9958
  - false_audio_56.wav: True: Incomplete, Predicted: Complete, Probability: 0.9180
  - false_audio_33.wav: True: Incomplete, Predicted: Complete, Probability: 0.9976
  - false_audio_36.wav: True: Incomplete, Predicted: Complete, Probability: 0.9961
  - false_audio_37.wav: True: Incomplete, Predicted: Complete, Probability: 0.9962
  - false_audio_39.wav: True: Incomplete, Predicted: Complete, Probability: 0.9550
  - false_audio_25.wav: True: Incomplete, Predicted: Complete, Probability: 0.9966
  - false_audio_40.wav: True: Incomplete, Predicted: Complete, Probability: 0.9915
  - false_audio_32.wav: True: Incomplete, Predicted: Complete, Probability: 0.9967
  - false_audio_38.wav: True: Incomplete, Predicted: Complete, Probability: 0.9757
  - false_audio_24.wav: True: Incomplete, Predicted: Compl

In [ ]:
from IPython.display import Audio
audio_path = '/content/mix_audios/true_audio_55.wav'
Audio(audio_path)

In [ ]:
from huggingface_hub import login
login()



In [ ]:
Step :9 deploy on huggingface

In [ ]:
from huggingface_hub import HfApi

api = HfApi()

repo_id = "vrushil/smart-turn-hindi-onnx-model"

api.create_repo(
    repo_id=repo_id,
    repo_type="model",
    exist_ok=True
)


RepoUrl('https://huggingface.co/vrushil/smart-turn-hindi-onnx-model', endpoint='https://huggingface.co', repo_type='model', repo_id='vrushil/smart-turn-hindi-onnx-model')

In [ ]:
from huggingface_hub import HfApi

api = HfApi()

onnx_fp32_path = "/data/output/v3.1-hf-custom-hindi/final_model/exports/model_fp32.onnx"
repo_id = "miraiminds/smart-turn-whisper-tiny-vaani-hindi"

api.upload_file(
    path_or_fileobj=onnx_fp32_path,
    path_in_repo="model.onnx",
    repo_id=repo_id,
    repo_type="model",
    commit_message="Upload FP32 ONNX Smart Turn model"
)

print("✅ Upload completed successfully")


Processing Files (0 / 0)      : |          |  0.00B /  0.00B            

New Data Upload               : |          |  0.00B /  0.00B            

  ...l/exports/model_fp32.onnx:   2%|1         |  551kB / 32.1MB            

✅ Upload completed successfully


optional for dataset

In [ ]:
# !pip install -q soundfile pyarrow


In [ ]:
# !pip install -q torchcodec


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.1/2.1 MB 25.8 MB/s eta 0:00:00


In [ ]:
# from datasets import load_dataset
# import os, csv
# from tqdm import tqdm
# import soundfile as sf

# BASE_DIR = "/content/smart_turn_hindi_10k"
# AUDIO_DIR = f"{BASE_DIR}/audio"
# os.makedirs(AUDIO_DIR, exist_ok=True)

# dataset = load_dataset(
#     "pipecat-ai/smart-turn-data-v3.1-train",
#     split="train",
#     streaming=True
# )

# TARGET = 10_000
# count = 0

# csv_file = open(f"{BASE_DIR}/train.csv", "w", newline="", encoding="utf-8")
# writer = csv.DictWriter(csv_file, fieldnames=[
#     "id","audio","endpoint_bool","language",
#     "midfiller","endfiller","synthetic",
#     "spoken_text","dataset"
# ])
# writer.writeheader()

# print("Saving Hindi samples → WAV + Smart-Turn CSV...")

# for sample in tqdm(dataset):
#     if sample["language"] != "hin":
#         continue

#     audio = sample["audio"]
#     arr = audio["array"]
#     sr = audio["sampling_rate"]

#     file_id = f"{count:06d}"
#     wav_path = f"{AUDIO_DIR}/{file_id}.wav"

#     sf.write(wav_path, arr, sr)

#     writer.writerow({
#         "id": file_id,
#         "audio": f"audio/{file_id}.wav",
#         "endpoint_bool": sample["endpoint_bool"],
#         "language": "hin",
#         "midfiller": False,
#         "endfiller": False,
#         "synthetic": False,
#         "spoken_text": None,
#         "dataset": "custom_v1"
#     })

#     count += 1
#     if count >= TARGET:
#         break

# csv_file.close()
# print("DONE — Hindi dataset created:", count)


/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


Resolving data files:   0%|          | 0/77 [00:00<?, ?it/s]

Resolving data files:   0%|          | 0/77 [00:00<?, ?it/s]

Saving Hindi samples → WAV + Smart-Turn CSV...


224345it [08:33, 436.57it/s]

DONE — Hindi dataset created: 10000


In [ ]:
# !pip install -q datasets huggingface_hub soundfile


In [ ]:
# import os
# os.chdir("/content/smart_turn_hindi_10k")   # <-- IMPORTANT

# from datasets import load_dataset, Audio

# ds = load_dataset("csv", data_files="train.csv")
# ds = ds.cast_column("audio", Audio(sampling_rate=16000))

# ds.push_to_hub("vrushil/smart-turn-data-10", private=False)


/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


Uploading the dataset shards:   0%|          | 0/4 [00:00<?, ? shards/s]

Map:   0%|          | 0/2500 [00:00<?, ? examples/s]

Creating parquet from Arrow format:   0%|          | 0/25 [00:00<?, ?ba/s]

Processing Files (0 / 0)      : |          |  0.00B /  0.00B            

...

Creating parquet from Arrow format:   0%|          | 0/25 [00:00<?, ?ba/s]

Processing Files (0 / 0)      : |          |  0.00B /  0.00B            

New Data Upload               : |          |  0.00B /  0.00B            

                              :   9%|8         | 41.9MB /  472MB            

CommitInfo(commit_url='https://huggingface.co/datasets/vrushil/smart-turn-data-10/commit/1c050f6c954de4c45d2229ec8507dc4442c2bca9', commit_message='Upload dataset', commit_description='', oid='1c050f6c954de4c45d2229ec8507dc4442c2bca9', pr_url=None, repo_url=RepoUrl('https://huggingface.co/datasets/vrushil/smart-turn-data-10', endpoint='https://huggingface.co', repo_type='dataset', repo_id='vrushil/smart-turn-data-10'), pr_revision=None, pr_num=None)